In [116]:
import yfinance as yf
import pandas as pd
from datetime import datetime

In [122]:
tickers = ['AAPL', 'MSFT']

company_profiles = []

for ticker in tickers:
    company = yf.Ticker(ticker)
    info = company.info
    
    
    # Extract relevant profile information
    profile = {
        'Ticker': ticker,
        'Name': info.get('shortName'),
        'Industry': info.get('industry'),
        'Sector': info.get('sector'),
        'Country': info.get('country'),
        'Business Summary': info.get('longBusinessSummary'),
        'Website': info.get('website'),
        'Market Cap': info.get('marketCap'),
        'Employees': info.get('fullTimeEmployees')
    }
    
    company_profiles.append(profile)

profiles_df = pd.DataFrame(company_profiles)
profiles_df.reset_index(drop=True, inplace=True)

profiles_df.head()
profiles_df.to_csv('data/company_profiles.csv', index=False)


In [118]:
profiles_df.head()

,Ticker,Name,Industry,Sector,Country,Business Summary,Website,Market Cap,Employees
0,AAPL,Apple Inc.,Consumer Electronics,Technology,United States,"Apple Inc. designs, manufactures, and markets ...",https://www.apple.com,2865636573184,150000
1,MSFT,Microsoft Corporation,Software - Infrastructure,Technology,United States,Microsoft Corporation develops and supports so...,https://www.microsoft.com,3173596266496,221000
2,GOOGL,Alphabet Inc.,Internet Content & Information,Communication Services,United States,Alphabet Inc. offers various products and plat...,https://abc.xyz,2153686106112,180895
3,TSLA,"Tesla, Inc.",Auto Manufacturers,Consumer Cyclical,United States,"Tesla, Inc. designs, develops, manufactures, l...",https://www.tesla.com,554091610112,140473
4,GM,General Motors Company,Auto Manufacturers,Consumer Cyclical,United States,"General Motors Company designs, builds, and se...",https://www.gm.com,49894178816,163000


In [119]:
start_date = '2023-01-01'
end_date = '2023-12-31'
all_tickers  = ['AAPL', 'MSFT', 'GOOGL','TSLA', 'GM', 'F', 'NVDA', 'DAL', 'AAL', 'LUV']

data_frames = []

for ticker in all_tickers:
    
    ticker_data = yf.Ticker(ticker).history(start=start_date, end=end_date)
    ticker_data['Ticker'] = ticker
    data_frames.append(ticker_data)

ticker_data_df = pd.concat(data_frames)
ticker_data_df.reset_index(drop=True, inplace=True)
ticker_data_df.dropna(inplace=True)

print(ticker_data_df.head())
ticker_data_df.to_csv('data/ticker_historical_data.csv')



         Open        High         Low       Close     Volume  Dividends  \
0  129.215478  129.830407  123.155402  124.048050  112117500        0.0   
1  125.853183  127.608724  124.057975  125.327515   89113600        0.0   
2  126.091219  126.725989  123.740589  123.998459   80962700        0.0   
3  124.980372  129.225391  123.869520  128.560867   87754700        0.0   
4  129.403910  132.319889  128.828647  129.086517   70790800        0.0   

   Stock Splits Ticker  
0           0.0   AAPL  
1           0.0   AAPL  
2           0.0   AAPL  
3           0.0   AAPL  
4           0.0   AAPL  


In [120]:

all_tickers = ['AAPL', 'MSFT', 'GOOGL', 'TSLA', 'GM', 'F', 'NVDA', 'DAL', 'AAL', 'LUV']

financial_data = []

for ticker in all_tickers:
    company = yf.Ticker(ticker)

    income_statement = company.financials.T  
    balance_sheet    = company.balance_sheet.T  
    
    income_statement = income_statement.reindex(index=income_statement.index.union(balance_sheet.index))
    balance_sheet    = balance_sheet.reindex(index=income_statement.index)
    
    # key metrics from financials and balance sheet
    combined_data = pd.DataFrame({
        'Year': income_statement.index,
        'Ticker': ticker,
        'Total Revenue': income_statement['Total Revenue'],
        'Gross Profit': income_statement['Gross Profit'],
        'Net Income': income_statement['Net Income'],
        'Total Debt': balance_sheet['Total Debt']
    })
    financial_data.append(combined_data)

financial_df = pd.concat(financial_data)
financial_df.dropna(inplace=True)
financial_df.reset_index(drop=True, inplace=True)
print(financial_df.head())
financial_df.to_csv('data/ticker_financial_data.csv', index=False)


        Year Ticker   Total Revenue    Gross Profit     Net Income  \
0 2020-09-30   AAPL  274515000000.0  104956000000.0  57411000000.0   
1 2021-09-30   AAPL  365817000000.0  152836000000.0  94680000000.0   
2 2022-09-30   AAPL  394328000000.0  170782000000.0  99803000000.0   
3 2023-09-30   AAPL  383285000000.0  169148000000.0  96995000000.0   
4 2020-06-30   MSFT  143015000000.0   96937000000.0  44281000000.0   

       Total Debt  
0  112436000000.0  
1  124719000000.0  
2  120069000000.0  
3  111088000000.0  
4   70998000000.0  


In [121]:

financial_df.columns


Index(['Year', 'Ticker', 'Total Revenue', 'Gross Profit', 'Net Income',
       'Total Debt'],
      dtype='object')

In [5]:
import requests
import pandas as pd

url = "https://senate-stock-watcher-data.s3-us-west-2.amazonaws.com/aggregate/all_transactions.json"
response = requests.get(url)
if response.status_code == 200:
    json_data = response.json()

    df = pd.json_normalize(json_data)
    allowed_columns = ['senator', 'ticker', 'owner', 'asset_description', 'asset_type', 'amount']
    
    df = df[allowed_columns]
    df.rename(columns={'senator': 'senator_name'}, inplace=True)
    
    # df['ticker'].replace('N/A', pd.NA, inplace=True)
    # df = df.dropna(subset=['ticker'])
    df =df[df['ticker'] != 'N/A']
    
    print(df.head())

           senator_name ticker   owner  \
0    Sheldon Whitehouse    ESS  Spouse   
1    Sheldon Whitehouse    ESS    Self   
3  Shelley Moore Capito    UPS  Spouse   
4  Shelley Moore Capito    MCD  Spouse   
5  Shelley Moore Capito    DUK  Spouse   

                                   asset_description asset_type  \
0            Essex Property Trust, Inc. Common Stock      Stock   
1            Essex Property Trust, Inc. Common Stock      Stock   
3           United Parcel Service, Inc. Common Stock      Stock   
4                McDonald's Corporation Common Stock      Stock   
5  Duke Energy Corporation (Holding Company) Comm...      Stock   

              amount  
0   $1,001 - $15,000  
1   $1,001 - $15,000  
3   $1,001 - $15,000  
4   $1,001 - $15,000  
5  $15,001 - $50,000  
